In [15]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [16]:
train_df = pd.read_csv('./data/train.csv.gz')

In [17]:
members_df = pd.read_csv('./data/members.csv.gz')
songs_df = pd.read_csv('./data/songs.csv.gz')

In [18]:
songs_extra_info_df = pd.read_csv('./data/song_extra_info.csv.gz')

In [19]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [20]:
songs_df.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [21]:
songs_extra_info_df.head()

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


In [22]:
train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


### Start By Seeing Correlations Between Where We View Song from and Replay
##### Note Nulls in Data - Will have to figure out what to do with them
    -source_system_tab
    -source_screen_name (Unknown)

In [45]:
train_df[["source_system_tab", "target"]].groupby(['source_system_tab'], as_index=True)\
.agg({'source_system_tab': 'count', 'target': 'mean'})\
.rename(columns={'target': 'mean_replay_chance', 'source_system_tab': 'count'})\
.sort_values(by='mean_replay_chance', ascending=False)

,count,mean_replay_chance
source_system_tab,,
my library,3684730,0.619659
settings,2200,0.590909
null,6478,0.433621
explore,167949,0.422146
search,623286,0.421362
discover,2179252,0.415770
notification,6185,0.378011
listen with,212266,0.326581
radio,476701,0.222662


In [41]:
train_df[["source_screen_name", "target"]].groupby(['source_screen_name'], as_index=True)\
.agg({'source_screen_name': 'count', 'target': 'mean'})\
.rename(columns={'target': 'mean_replay_chance', 'source_screen_name': 'count'})\
.sort_values(by='mean_replay_chance', ascending=False)

,mean_replay_chance,count
source_screen_name,,
Payment,0.666667,12
My library,0.657173,75980
Local playlist more,0.636983,3228202
My library_Search,0.611223,6451
Discover Chart,0.517032,213658
Concert,0.510638,47
Search,0.471749,298487
Discover New,0.455531,15955
Explore,0.448149,72342


In [46]:
train_df[["source_type", "target"]].groupby(['source_type'], as_index=True)\
.agg({'source_type': 'count', 'target': 'mean'})\
.rename(columns={'target': 'mean_replay_chance', 'source_type': 'count'})\
.sort_values(by='mean_replay_chance', ascending=False)

,count,mean_replay_chance
source_type,,
local-playlist,1079503,0.657719
local-library,2261399,0.632098
artist,3038,0.572745
topic-article-playlist,11194,0.494283
song,244722,0.437460
online-playlist,1967924,0.424985
top-hits-for-artist,423614,0.418537
album,477344,0.393421
song-based-playlist,210527,0.380460
